In [ ]:
import pandas as pd 
import re
import ast
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve,ConfusionMatrixDisplay,RocCurveDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stat
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
stimulus=pd.read_excel("Stimulus_Description.xlsx")
stimulus["Target Emotion"]=stimulus["Target Emotion"].str.title()
stimulus.info()
stimulus.head()

In [ ]:
stimulus=pd.read_excel("Self-annotation Multimodal_Use.xlsx")
stimulus.info()
stimulus.head()

In [ ]:
# For first run only
gsr_data=pd.read_excel("Self-annotation Multimodal_Use.xlsx")
gsr_data["filename"]="GSRdata_s"+gsr_data["Session ID"].astype(str)+"p"+gsr_data["Participant Id"].astype(str)+"v"+gsr_data["Video ID"].astype(str)+".dat"
PATH_2="GSR/"
gsr_data["filename"]=gsr_data["filename"].str.replace('GSRdata_s2p9v3.dat','GSRdata_S2p9v3.dat',regex=False)
gsr_data["GSR_list"]=gsr_data["filename"].apply(lambda x: list(pd.read_table(PATH_2+x).iloc[:,0]))
gsr_data=gsr_data.merge(stimulus.iloc[:,0:3],on=["Session ID","Video ID"])
gsr_data.to_csv("gsr_data.csv",index=False)

In [ ]:
gsr_data=pd.read_csv("gsr_data.csv")
gsr_data.info()
gsr_data.head()

In [ ]:
# For first run only
gsr_summary=gsr_data[["Emotion","GSR_list"]].copy()
gsr_summary["GSR_list"]=gsr_summary["GSR_list"].apply(lambda x: ast.literal_eval(x))
gsr_summary["min"]=gsr_summary["GSR_list"].apply(lambda x: np.min(x))
gsr_summary["max"]=gsr_summary["GSR_list"].apply(lambda x: np.max(x))
gsr_summary["mean"]=gsr_summary["GSR_list"].apply(lambda x: np.mean(x))
gsr_summary["variance"]=gsr_summary["GSR_list"].apply(lambda x: np.var(x))
gsr_summary["kurtosis"]=gsr_summary["GSR_list"].apply(lambda x: stat.kurtosis(x))
gsr_summary["skewness"]=gsr_summary["GSR_list"].apply(lambda x: stat.skew(x))
gsr_summary["first_quartile"] = gsr_summary["GSR_list"].apply(lambda x: np.percentile(x, 25))
gsr_summary["third_quartile"] = gsr_summary["GSR_list"].apply(lambda x: np.percentile(x, 75))
gsr_summary["median"] = gsr_summary["GSR_list"].apply(lambda x: np.median(x))
gsr_summary=gsr_summary.drop(columns=["GSR_list"])
gsr_summary.to_csv("gsr_summary.csv",index=False)

In [ ]:
gsr_summary=pd.read_csv("gsr_summary.csv")
gsr_summary.info()
gsr_summary.head()

In [12]:
# For first run only
gsr_proc=gsr_data[["Emotion","GSR_list"]].copy()
gsr_proc["GSR_list"]=gsr_proc["GSR_list"].apply(lambda x: ast.literal_eval(x))
gsr_proc=pd.concat([gsr_proc,pd.DataFrame(gsr_proc['GSR_list'].tolist())],axis=1).drop(columns=["GSR_list"])
for i in range(gsr_proc.shape[1]-1):
    gsr_proc[i]=pd.to_numeric(gsr_proc[i])
gsr_proc.to_csv("gsr_proc.csv",index=False)

In [ ]:
gsr_proc=pd.read_csv("gsr_proc.csv")
gsr_proc.info()
gsr_proc.head()

In [ ]:
def plot_signals(data_arr, title = ''):
    plt.clf()
    plt.figure(figsize=(10, 4))
    
    for index, row in data_arr.iterrows():
        y = row
        plt.plot(y)
    
    plt.tight_layout()
    plt.title(title)
    plt.ylabel("Skin conductivity")
    plt.xlabel("Record index")
    plt.show()

for i in emos:
    plot_signals(gsr_proc[gsr_proc["Emotion"]==i].iloc[:,1:4998:400], "GSR Signal - "+i)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
file_path = 'gsr_proc.csv'  # Replace with the path to your CSV file
gsr_proc = pd.read_csv(file_path)

# Extract the first 'Happy' row
happy_row = gsr_proc[gsr_proc["Emotion"] == "Happy"].iloc[0]

# Drop the 'Emotion' column
happy_values = happy_row.drop(labels="Emotion").values.astype(float)

# Generate time stamps
time_stamps = range(len(happy_values))

# Plot the graph
plt.figure(figsize=(10, 5))
plt.plot(time_stamps, happy_values, label='Happy')
plt.xlabel('Time Stamp')
plt.ylabel('GSR Value')
plt.title('GSR Values for Happy Emotion Over Time')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
file_path = 'gsr_proc.csv'  # Replace with the path to your CSV file
gsr_proc = pd.read_csv(file_path)

# Extract the first 'Happy' row
happy_row = gsr_proc[gsr_proc["Emotion"] == "Sad"].iloc[0]

# Drop the 'Emotion' column
happy_values = happy_row.drop(labels="Emotion").values.astype(float)

# Generate time stamps
time_stamps = range(len(happy_values))

# Plot the graph
plt.figure(figsize=(10, 5))
plt.plot(time_stamps, happy_values, label='Sad')
plt.xlabel('Time Stamp')
plt.ylabel('GSR Value')
plt.title('GSR Values for Happy Emotion Over Time')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from itertools import cycle

# Load the dataset
gsr_summary = pd.read_csv("gsr_summary.csv")

# Define features and target variable
X = gsr_summary.drop(columns=["Emotion"])
y = gsr_summary["Emotion"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define classifiers
classifiers = {
    'Decision Tree': DecisionTreeClassifier(random_state=3),
    'Random Forest': RandomForestClassifier(random_state=3),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=3),
    'SVM': SVC(probability=True, random_state=3),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'AdaBoost': AdaBoostClassifier(random_state=3),
    'Gradient Boosting': GradientBoostingClassifier(random_state=3),
    'MLP': MLPClassifier(max_iter=1000, random_state=3),
    'Ridge Classifier': RidgeClassifier()
}

# Initialize dictionaries to store results
accuracy_scores = {}
precision_scores = {}
recall_scores = {}
f1_scores = {}
roc_aucs = {}
fprs = {}
tprs = {}

# Binarize the output
y_train_bin = label_binarize(y_train, classes=np.unique(y))
y_test_bin = label_binarize(y_test, classes=np.unique(y))
n_classes = y_test_bin.shape[1]

# Train and evaluate each classifier
for clf_name, clf in classifiers.items():
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    
    # Calculate metrics
    accuracy_scores[clf_name] = accuracy_score(y_test, y_pred)
    precision_scores[clf_name] = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall_scores[clf_name] = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1_scores[clf_name] = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    
    # Calculate ROC curve and AUC
    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test_scaled)
    else:
        y_score = clf.decision_function(X_test_scaled)
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Compute micro-average ROC curve and ROC area
    fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
    roc_auc_micro = auc(fpr_micro, tpr_micro)
    
    roc_aucs[clf_name] = roc_auc_micro
    fprs[clf_name] = fpr_micro
    tprs[clf_name] = tpr_micro

# Print metrics
print("Accuracy Scores:")
for clf_name, acc in accuracy_scores.items():
    print(f"{clf_name}: {acc:.4f}")

print("\nPrecision Scores:")
for clf_name, prec in precision_scores.items():
    print(f"{clf_name}: {prec:.4f}")

print("\nRecall Scores:")
for clf_name, rec in recall_scores.items():
    print(f"{clf_name}: {rec:.4f}")

print("\nF1 Scores:")
for clf_name, f1 in f1_scores.items():
    print(f"{clf_name}: {f1:.4f}")

# Plot AUC curve
plt.figure(figsize=(14, 7))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'blue', 'green', 'red', 'purple', 'yellow', 'cyan', 'magenta'])

for clf_name, color in zip(classifiers.keys(), colors):
    plt.plot(fprs[clf_name], tprs[clf_name], color=color, lw=2,
             label=f'{clf_name} (AUC = {roc_aucs[clf_name]:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()